In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tqdm import tqdm
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
from sklearn.model_selection import train_test_split
import gc
from tokenizers import ByteLevelBPETokenizer, processors

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\hvutr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(device)


if not torch.cuda.is_available():
    print("CUDA is not available on your system.")
else:
    # Print the number of CUDA devices
    print(f"Number of CUDA Devices: {torch.cuda.device_count()}\n")

    for i in range(torch.cuda.device_count()):
        print(f"Device {i}:")
        print(f"    Name: {torch.cuda.get_device_name(i)}")
        print(f"    Computational Capability: {torch.cuda.get_device_capability(i)}")
        print(f"    Total Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**2):.2f} MB\n")



cuda
Number of CUDA Devices: 1

Device 0:
    Name: NVIDIA GeForce RTX 3090
    Computational Capability: (8, 6)
    Total Memory: 24575.50 MB



In [3]:
data_path = 'datasets/tiny_stories/'

In [4]:
filename = 'tiny_stories.csv'
filepath = os.path.join(data_path, filename)

data = pd.read_csv(filepath)
data = data.dropna()

print(data.head())


   Unnamed: 0                                                 en  \
0           0  Once upon a time, there was a little boy named...   
1           1  Once upon a time, there was a normal boy named...   
2           2  Once upon a time, there was a fast car named S...   
3           3  Once there was a family with a very playful go...   
4           4  Once upon a time, there was a dog named Max. M...   

                                                  vi  
0  Ngày xửa ngày xưa, có một cậu bé tên Tim. Tim ...  
1  Ngày xửa ngày xưa, có một cậu bé bình thường t...  
2  Ngày xửa ngày xưa, có một chiếc xe tốc độ tên ...  
3  Có một gia đình có một con dê rất hay vui đùa....  
4  Ngày xửa ngày xưa, có một con chó tên là Max. ...  


In [5]:
N = len(data)
train_size = int(0.9 * N)
train_data = data[:train_size]
val_data = data[train_size:]

print(train_data.shape, val_data.shape)

(2339822, 3) (259981, 3)


In [8]:
from transformers import BertTokenizer, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Get the IDs
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
unk_id = tokenizer.unk_token_id
pad_id = tokenizer.pad_token_id

print("PAD Token ID:", pad_id)
print("UNK Token ID:", unk_id)
print("CLS Token ID:", cls_id)
print("SEP Token ID:", sep_id)

C:\Users\hvutr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hvutr\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

PAD Token ID: 1
UNK Token ID: 3
CLS Token ID: 0
SEP Token ID: 2


In [9]:
BLOCK_SIZE = 256
VOCAB_SIZE = tokenizer.vocab_size
print('Vocab size is ', VOCAB_SIZE)


Vocab size is  64000


In [14]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    samples = data['vi'].sample(n=batch_size)

    for i, sample in enumerate(samples):
        summary_ids = tokenizer.encode(sample)
        summary_ids = [id for id in summary_ids if id != unk_id]
        if len(summary_ids) < block_size + 2:
            summary_ids = summary_ids + [pad_id] * (block_size + 2 - len(summary_ids))
        random_start = random.randint(0, len(summary_ids) - block_size - 2)
        x[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start:random_start + block_size], dtype=torch.long)
        y[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start + 1:random_start + block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist(), skip_special_tokens=False))
print(tokenizer.decode(b[0].tolist(), skip_special_tokens=False))

torch.Size([1, 256]) torch.Size([1, 256])
<s> Ngày xửa ngày xưa, có một con dê bé tên là Billy, rất bướng bỉnh. Thằng bé thích ăn lúa mạch cả ngày. Một hôm, nó đi đến kho thóc đỏ lớn để kiếm ít yến mạch.Khi Billy đến kho chứa cỏ, cậu bé nhìn thấy một cái túi to đựng đầy yến mạch. 'Chà' Billy nói, "Tôi muốn ăn hết chỗ yến mạch này" Nhưng cái bao quá nặng nên cậu bé không mở được. Cậu cố gắng và cố gắng, nhưng cái bao vẫn không chịu mở ra.Ngay sau đó, một chú chuột thân thiện tên Tim đi ngang qua. "Chào Billy," Tim nói. 'Anh có cần giúp đỡ không?' Billy gật đầu và nói, 'Vâng, làm ơn! Tôi không thể mở cái túi yến mạch to này được.' Tim dùng hàm răng sắc nhọn của mình để mở chiếc túi, và cả hai cùng nhau nhồi đầy bụng với những viên yến mạch ngon lành. Từ ngày hôm đó trở đi, Billy học được rằng nhờ giúp đỡ khi cần là điều bình thường, và cậu và Tim trở thành những người bạn tốt nhất. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Ngày xửa ngày xưa, có một con dê bé tên là Billy, rất 

In [ ]:
del model, xb, yb  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

In [18]:
N_EMB = 800
N_LAYERS = 4
N_HEADS = 4
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.block_size = block_size

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 4 * n_emb),
            nn.ReLU(),
            nn.Linear(4 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device))

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == sep_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

Number of parameters 131176992


In [10]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_4layers.pth")
print(model_path)
model = torch.load(model_path)

datasets/tiny_stories/english_tiny_stories_6layers.pth


In [19]:
EARLY_STOP = 50
N_EPOCHS = 5000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

last_val_loss = 1e9
early_stop = EARLY_STOP

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
        # if val_loss >= last_val_loss:
        #     early_stop -= 1
        #     if early_stop == 0:
        #         print('Early stop!')
        #         break
        # else:
        #     early_stop = EARLY_STOP
        #     last_val_loss = val_loss

Step: 0 Training Loss: 11.021085739135742
Validation loss: 9.176621437072754
Step: 100 Training Loss: 3.6481001377105713
Validation loss: 3.4954440593719482
Step: 200 Training Loss: 2.870316982269287
Validation loss: 3.0306267738342285
Step: 300 Training Loss: 2.8013927936553955
Validation loss: 2.9989569187164307


In [49]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_4layers.pth")
torch.save(model, model_path)

In [48]:
starting_tokens = 'Một ngày đẹp trời,'
encoded_start = tokenizer.encode(starting_tokens)
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 10
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=2000, temperature=0.5, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation, skip_special_tokens=False)

    print('Story ', _ + 1, ':')
    print(story)
    print('\n')






Story  1 :
[CLS] The dog chased the cat. The cat was scared but tired of running. The cat ran and ran until it was very tired. It curled up in a tree and fell asleep. The cat had a little nap. In its nap, the cat had a dream. In its dream, it saw a big, soft pillow. The cat slept there, and when it woke up, it looked down at the pillow. The cat was very happy. It had a new friend. The cat slept all day. [SEP]


Story  2 :
[CLS] The dog chased the cat. The cat was scared but tired of running. Suddenly, it saw a big tree. The cat was curious and decided to climb it. But the cat was too fast and too fast. The cat was too fast and it hit the tree. The cat fell down and landed on the ground. The cat was hurt and sad. The cat felt bad and said sorry. The cat said sorry and said sorry. The cat learned to be more careful next time. [SEP]


Story  3 :
[CLS] The dog chased the cat. The cat was scared but tired of running. He ran and ran, but he couldn't catch it. He was getting very tired and sa